In [30]:
import sys
sys.path.append('../')
from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib3
from utils.data_gatherer import *
from tqdm import tqdm
import pickle

#### get tickers list

In [31]:
http = urllib3.PoolManager()
ticker = []
web_id = []
name = []

look_up = pd.DataFrame({'Ticker':[],'Name':[],'WEB-ID':[],'Market':[]})
#bourse tickers
r = http.request('GET', "http://old.tsetmc.com/Loader.aspx?ParTree=15131J&i=32097828799138957") 
soup = BeautifulSoup(r.data.decode('utf-8'), 'html.parser')
table = soup.findAll("table", {"class": "table1"})
stock_list = table[0].find_all('a')
for stock in stock_list:
    ticker.append(stock.text)
    name.append(stock.attrs["title"])
    web_id.append(stock.attrs["href"].split("&i=")[1])
bourse_lookup = pd.DataFrame({'Ticker':ticker, 'Name':name,'WEB-ID':web_id}) 
bourse_lookup['Market'] = 'بورس'
look_up = pd.concat([look_up,bourse_lookup],axis=0)

#farabourse tickers
r = http.request('GET', 'http://old.tsetmc.com/Loader.aspx?ParTree=15131J&i=43685683301327984') 
soup = BeautifulSoup(r.data.decode('utf-8'), 'html.parser')
table = soup.findAll("table", {"class": "table1"})
stock_list = table[0].find_all('a')
ticker = []
web_id = []
name = []
for stock in stock_list:
    ticker.append(stock.text)
    name.append(stock.attrs["title"])
    web_id.append(stock.attrs["href"].split("&i=")[1])
farabourse_lookup = pd.DataFrame({'Ticker':ticker, 'Name':name,'WEB-ID':web_id}) 
farabourse_lookup['Market'] = 'فرابورس'
look_up = pd.concat([look_up,farabourse_lookup],axis=0)

# rearrange columns
look_up['Ticker'] = look_up['Ticker'].apply(lambda x: (str(x).replace('ي','ی')).replace('ك','ک'))
look_up['Ticker'] = look_up['Ticker'].apply(lambda x: x.replace('\u200c',' '))
look_up['Name'] = look_up['Name'].apply(lambda x: (str(x).replace('ي','ی')).replace('ك','ک'))
look_up['Name'] = look_up['Name'].apply(lambda x: x.replace('\u200c',' '))

In [32]:
look_up.shape

(573, 4)

In [33]:
look_up.head()

,Ticker,Name,WEB-ID,Market
0,آباد,توریستی ورفاهی آبادگران ایران,59612098290740355,بورس
1,آبادا,تولید نیروی برق آبادان,37661500521100963,بورس
2,آپ,آسان پرداخت پرشین,55254206302462116,بورس
3,آسیا,بیمه آسیا,51106317433079213,بورس
4,اپال,فرآوری معدنی اپال کانی پارس,655060129740445,بورس


#### fetch data

In [34]:
def get_tse_ticker_ohlc(ticker):
    filtered_columns = ['Date', 'Adj Open', 'Adj High','Adj Low', 'Adj Close']
    price_data = get_daily_price_data(symbol=ticker, start_date='1398-01-01', end_date='1402-10-01')[filtered_columns]
    renamed_dict = {'Adj Open': 'open',
                'Adj High': 'high',
                'Adj Low': 'low',
                'Adj Close': 'close'}
    price_data = price_data.rename(renamed_dict, axis=1).reset_index()
    price_data.columns = [col.lower() for col in price_data.columns]
    price_data['date'] = pd.to_datetime(price_data['date'])
    return price_data

In [35]:
all_tickers = list(look_up['Ticker'].unique())
error_tickers = []
dict_price_data = {}
    

for i in tqdm(range(len(all_tickers)), desc="Downloading..."):
    try:
        ticker = all_tickers[i]
        dict_price_data[ticker] = get_tse_ticker_ohlc(ticker)
        
    except:
        error_tickers.append(ticker)
        print(f'Error in download: {ticker}')
        continue

Downloading...:   0%|          | 0/573 [00:00<?, ?it/s]

Downloading...:   2%|▏         | 13/573 [00:59<46:30,  4.98s/it]/Users/aaa/Desktop/ph.D/term1/Echometrics/final_projects/data/../utils/data_gatherer.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  symbol_id = search_ticker_webid(symbol).loc[symbol, 'WEB-ID'].values[0]
Downloading...:   6%|▌         | 32/573 [02:17<28:34,  3.17s/it]/Users/aaa/Desktop/ph.D/term1/Echometrics/final_projects/data/../utils/data_gatherer.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  symbol_id = search_ticker_webid(symbol).loc[symbol, 'WEB-ID'].values[0]
Downloading...:   8%|▊         | 46/573 [03:13<32:55,  3.75s/it]

stock is not founded!
Error in download: تکاردان


Downloading...:   8%|▊         | 47/573 [03:14<25:13,  2.88s/it]

stock is not founded!
Error in download: تکشا


...

Downloading...:  85%|████████▍ | 486/573 [25:43<03:27,  2.38s/it]

stock is not founded!
Error in download: شرانل


Downloading...:  85%|████████▌ | 489/573 [25:48<02:40,  1.91s/it]

stock is not founded!
Error in download: شگویا


Downloading...:  87%|████████▋ | 496/573 [26:04<03:02,  2.37s/it]/Users/aaa/Desktop/ph.D/term1/Echometrics/final_projects/data/../utils/data_gatherer.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  symbol_id = search_ticker_webid(symbol).loc[symbol, 'WEB-ID'].values[0]
Downloading...:  88%|████████▊ | 503/573 [26:21<02:47,  2.39s/it]/Users/aaa/Desktop/ph.D/term1/Echometrics/final_projects/data/../utils/data_gatherer.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  symbol_id = search_ticker_webid(symbol).loc[symbol, 'WEB-ID'].values[0]
Downloading...:  97%|█████████▋| 556/573 [28:29<00:39,  2.31s/it]/Users/aaa/Desktop/ph.D/term1/Echometrics/final_projects/data/../utils/data_gatherer.py:12: PerformanceWarning: indexing past lexsort depth may impact performance.
  symbol_id = search_ticker_webid(symbol).loc[symbol, 'WEB-ID'].values[0]
Downloading...: 100%|██████████| 573/573 [29:13<00:00,  3.06s/it]


#### dump as pickle file

In [36]:
with open('tse_symbols_price.pickle', 'wb') as handle:
    pickle.dump(dict_price_data, handle)